## Badge encodings tutorial

See the examples below for common ways to map data to node badges in Graphistry. Icons appear in the main area of a node, while badges circle them (`TopRight`, `BottomLeft`, `Right`, etc.). They can be used together.

Badges are quite configurable. You can set the glyph, color, border, and define their encoding to be based on the ndoe data.

The [PyGraphistry Python client](https://github.com/graphistry/pygraphistry) makes it easier to use the [URL settings API](https://hub.graphistry.com/docs/api/1/rest/url/) and the [REST upload API](https://hub.graphistry.com/docs/api/2/rest/upload/). For dynamic control, you can use also use the [JavaScript APIs](https://hub.graphistry.com/docs/).

## Setup

Mode `api=3` is recommended. It is required for `complex_encodings` (ex: `.encode_point_size(...)`). Mode `api=1` works with the simpler `.bind(point_size='col_a')` form.

In [ ]:
# ! pip install --user graphistry
import graphistry

# To specify Graphistry account & server, use:
# graphistry.register(api=3, username='...', password='...', protocol='https', server='hub.graphistry.com')
# For more options, see https://github.com/graphistry/pygraphistry#configure

graphistry.__version__

In [ ]:
import datetime, pandas as pd
e_df = pd.DataFrame({
    's': ['a', 'b', 'c', 'a', 'b', 'c', 'a', 'd', 'e'],
    'd': ['b', 'c', 'a', 'b', 'c', 'a', 'c', 'e', 'd'],
    'time': [datetime.datetime(1987, 10, 1), datetime.datetime(1987, 10, 2), datetime.datetime(1987, 10, 3),
             datetime.datetime(1988, 10, 1), datetime.datetime(1988, 10, 2), datetime.datetime(1988, 10, 3),
             datetime.datetime(1989, 10, 1), datetime.datetime(1989, 10, 2), datetime.datetime(1989, 10, 3)]
})
n_df = pd.DataFrame({
    'n': ['a', 'b', 'c', 'd', 'e'],
    'score': [ 1, 30, 50, 70, 90 ],
    'palette_color_int32': pd.Series(
        [0, 1, 2, 3, 4],
        dtype='int32'),
    'hex_color_int64': pd.Series(
        [0xFF000000, 0xFFFF0000, 0xFFFFFF00, 0x00FF0000, 0x0000FF00],
        dtype='int64'),
    'type': ['mac', 'macbook', 'mac', 'macbook', 'sheep'],
    'assorted': ['Canada', 'mac', 'macbook', 'embedded_smile', 'external_logo'],
    'origin': ['Canada', 'England', 'Russia', 'Mexico', 'China']
    
})

g = graphistry.edges(e_df, 's', 'd').nodes(n_df, 'n')

### Badges as categorical mappings + glyph types

The most common form is mapping distinct values to glyphs. 

Graphistry supports built-in + custom glyphs:

* Built-in general glyphs: Use values from [Font Awesome 4](https://fontawesome.com/v4.7.0/) or, more explicitly, `fa-thename`
* Built-in flag icons: Use ISO3611-Alpha-2 values
* Custom image URL
* Custom image data URI (embedded)

In [ ]:
g.encode_point_badge(
  'assorted',
  'TopRight',
  shape="circle", #clip excess
  categorical_mapping={
      'macbook': 'laptop', #https://fontawesome.com/v4.7.0/icons/
      'Canada': 'flag-icon-ca', #ISO3611-Alpha-2: https://github.com/datasets/country-codes/blob/master/data/country-codes.csv
      'embedded_smile': 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAARkAAAC0CAMAAACXO6ihAAAAgVBMVEUAAAD////T09P4+PiYmJihoaHf39/7+/vm5uYLCwsRERHy8vJ+fn7v7+83Nze2trbZ2dlRUVFGRka9vb1xcXEYGBhbW1tWVlbq6uq7u7usrKxgYGA8PDwWFhbNzc14eHhpaWmoqKiPj48vLy8oKCggICCIiIiUlJRBQUFKSkokJCTGfDo0AAAOxklEQVR4nO1d6YKyOgxFNkFEUFwQFFEUdd7/AS9L06ZlkZm5I37i+acUSA9dkjRNJal/JPNYjfxRBlvW1guzb3leBPupJo8wLGfat0wvgaOrjEQo0b1vsfrGTKvSUiLtW7RekaRNvGRQV32L1x9it5mX0Wh86lu+3uCM24jJEPYtYT+468Kg6xuGz3cuu28Ze0FqYA7c4/Wwud3WUy3ihpq+pewBB9yTnPWMXjivfURYjxL2hIDV3j+KFx3Wwwan8TFixl5VpdtvswuWHEXGKOhBuD4xZS0mntRcPxvRYT1Lkvt0OSwLiql38qy+xP65Ar0K5nRWCgas5dZgQvWYtkEkdDzPG5gSTGeeoHkMmRDzWx+SFhx3aDGhRbXg+GmC9Y0E+pKbNJa5IfXYf6Js/eIEVW4mRuIMhKH0pztM2MvmMjtMzEh+nnC9QiP1jb6ayzgcM0PpTsSOVA4tZfQhMgNmQWsX4dwTI/1ZsvUL8G7GbYV8jplhuGjupNJKayl+7en8JNn6hUpq67WWWmJitk8SrWfArPNAR0Hdybg9R7KecQYVbtFebsuI2TxHsr5xh1mnvSEwTc9qUZTfCiHRZqwGf1WJxB7YvCQhM7utzczQ1BQ/SbDesezCDLYN5k8TrWdQZprNyT0mRm/tde8E2psaF9hmnJbX4vR7M2zo2NrQnea8MTmYARi1iKj2uhBMMx7QAiVbmnSqHSXmXHkZjAEFpF1ZtR3hUlgNpqlvWO+JBLle3DW6EMqWyMuwmOEdmXYabubzTZjy3ciFYXg4A3AOMWBRqUQwOmfCjPLA7nwzpNVOwyOV9p3cW+8HtZWXKFN0DqQZGX2L+mRMtGZe3CI4GqZ2rW9Rn45rIy+lN6aj4+8dsa6Lj9bjS3k1AUV5MOYkRhjhNaWxLmtsyfLuyZHjOMdTXZjaAHBPtSCSM3ialg5ren4M85xk+FY82uoTvVaDS1z4MOy2FfE3xuoWp6qqaiVU7TQNb+V4k1ATQhvcALRfOrJsWLzJrdiGHMmOGiKf1qDiGs93zX+0pYfCH4zjXIqvxmM+EIYy1MRRjW+mFcOwG9Z6y07KBoj+wHfEWvT9ftpMgUnw3X5U4u3XFMSNlARjX/e2GVyjibe4b8n/GBVHja9H14Wgxu3n6dbV+SQA2/deutxvBVq2aksU0QkpO9Z7O272HsdLdHigvSGXznuPMgmek6wILXObl2S2u1VtatAFlfh5UvaAL0QMzhoyWV4D2bdqx1g1shU7Ct7c04e6ko5jaWZkQcWu61tmuAzfPTgYbdbmd1LeYdhp2a/xzqBbnEa2EHwFoUeVve3DwJkGQlfmX2hMw1rRpmCjb0WBIZxZLdvC3hg0WK9mmCVX3PdWchuwAp1NqTaMBTDTg1z9YwomUI0rAaIBhuB/qYCaS1bNRVi3fW/DqAFzaDJx9doKvBLPFuolcABdrmYXytJqbk7vD+gwdRrLEfTip0v1CiCVrwv1ncGsNZzlJASTVL5uVzIoOv7u6WK9AEJS+7otpmCAD9NogtDOmqEE9gi2bv9/XwTN7QLMKXlwoQ4FnMY2swQf+EDtbND/K1sK6MQ0TGWG7RfURbcl3ZAwqOAYhITUfywkjKZB98ZgI/GAAZdbA6AG+JsvJrWBLmVvGTUTlRIzSP9DCZax04XmsWZrLNaAI4MXLL5BMZw4uasyS3YwTMcMoG0Xz7Dz+M/qg2by+Sq49C1cv9g0MeMMnBghWRNrMQOeligOdpUYf7iKDMbGE4lxBpK16SH2sY7Cy8bRepCLkg0w1Uh2DUOPInWgScfbcFkshpLi64MPPvjggw8++OCDDz744IMPPvjggw8++OCDDz744IM/wWTF0FrwHERe/ByZXgNbxQa07vc6Fou90ZunPsDAqVZaikGgljycEJtuzFxo7OdwNnJ3Y+ZOywzlQFMuqUgLMwdaZiCHU2ZII5ZmsLnUENuMeTlPrYfMnCkzncaZxNd14w12dpgddmlD0KPcab9Gse/lDbIRdmHGTIssI1634HtlQMxI0nyapnE3beY4GhQz3bH2/4iZr83ykGE5b0m2tciKLDeVeMxbXNwZtpwceAmLW/EmUp6ZW/72uGP0XhIWL7zj0WdGdqq2MbMp7ur2DoDmRb4yHo8VxZA9Jy7/3J1UQP7PMtKzIooRBZiCdeDp9ji/1dY9TyV1ozeq+W6LtRPZ5a0aqztm5hS5+dtt2UPxwXP2EBX3p+lWtgtZ9chLIT50Cfsxo2txg5Y9aEZv13KJr5FR3nXMnhayZ5Paxuwfuj/6HAjB3YpV2HAHtvPGkxKPRfJaNF717gtZfi3jlL13z8rakrlFZey4yswdJVpU2L5lljAMn5sR2viN4zJboWFXkg1P8Z6qUcrtCIlMrIST8H6UuA30SvQA9MopOxwng7fBebEV0ozPlZjwHFnd9uyXfRMyasOWHcrMkk/tGpktzHzVvPEq3WqkCHlmQvzxZU4JJ8ygw7fK04C+6s/IUXhmZG4PDmHmLFduy6FxzCjiYSqQzYkyI+5G0JqZ2dclXD6yFFoIc44Zh/s83ZhpODxIYEZIuV8yU3eAzEhkpgr9xjNTefWlkZna87GOtbvKFhwzfGfrxAzK/qfngJx3OTPLmr0l5cWCGfb9jOJWlMhqX+n2mNpTlRnbMFDXbWQGfupemgYukfUo1bRda4cNUgEyx0N1nMmZgaxkI8Ur9zLeSH1zZmZTVcPtwgucUpiCGdqijFMh9+VIap/3hvSqoa/iBupJY0T6e4EZVwtXyXIL7BHz8a4GHrxjxnGllTNAaBBmrk4QwOPdoEA+myWnq4Y+rr0NiBmbvWCtasAnYSZk/+TMgPhjmpVBZcwUwzP9lO5yBc05Z+YMnPmgIEBRMk7MqEhBkVjmwtrAjGcmIolnoHI6NQZWEb4DCthwOSDM5ICxRtBnWEeL5hM4d7ikHjbpsa1nS3YeGx2xWSYLjWdmDhOHXogfUmbW8MYY7hSYgWQ8LBXGkfuHMqNDGtMV+QOlYoQvxzFDk0tgZsD4FJiZU2L2rHZyMf1Bf2HMEDU6ZwYYRcf5NTFTDg4kw86KfWD2XIGZCSnAdvXPYYLQOWbYzlvSndC5txwzE+jpcLVoAYrahRmSKrQrMzTJHXpcAzNEmjg/C0bPVY4xX66ZGZSmCOppccyw63I3ZmgrXJ7SdLrcdWHGK4emoJC/bGVtzNygM6Gd5A3MkL69uu1myfnC3G0oOchjZo7kL9+sZ8ZrZ4bNQV41JdYDZkj1z4X8q4fMwG0+8n80MCOm1AQ1CLkhHzMD05ny9TNm0HQnT2OenW7MYLQxA6OzjFa6OjJTHWY6MLODrjn/GTP8vmY3wPt2/19mYKbDSa47MgMNGyUueMzMhPReZfMzZiTh8AwF7Wl+OWbQCNWBGTjZ9qfMzCrnihggwKswQ6eJ7zFj/JIZaVMxtuzwT5lBvrS/ZEaC3nT7KTPSLhUPT9P/ghkYgW3kAezGjPkjZjaQ1HX/Y2YyxWGq85au8wfMUNUZHSB/7MQM9UCgxImPmYHvXShHP2VGyt2w24iZ5tHld8ywI35iG5jZgXBo+oOD0Tsyg5IqPmYGTP8mHbgrMxnOMVVuSgfqz5mJqLIKpzkb0oQTtcSoGzNgV3xH07tAF4x+xswsiFzbkol/3gT3y/jwO2Ys8MpT+Qx05DWsBl9gNn7EDPjLdDatpcg/U8sM9STFP2NmWrzAoCsXnE71fWZuY3y/hNybGTN3egZQWnS2TQSlHzFzgQ/GVD3hTPUKM3TUHp1/yEzRKHXKDLxwipkRcqa2MJNQD1Ph9dgwp17uuaLCjg1HDVC6MmAmVOqZAVt75MKMP0fezlpmqBOp/KcmL3uVGei0CWKG2ktwgE+MmZH5lUT4uy4zEm0jlh5F1M9LmBH1bYGZVQKi6bNFZqSix1L3s15M+UlIbzUWixliZuQu57fkNqcj5jiviXmmaQazJxcN9rIA/4x6u+XzjXmhp96Gi90KBrKg/BYJ+AijYuWTvj/YTcxZKBf0r2jCVG+XyS+sjx+azpUVPOQ8Cr9a6jE3sG3rMjb+Z+z8mKPKXKgF/PwYkAn6w5DRqoa3ynh12A3ZkzMLc+NQp29+HPA1q6vj0TZs2caJDvGuli87UtFKfyTVQEa+4+TX8sMdPSrk2PZdWWg4l8oajWIxZqRjPTN+puKY4rKJhXOoNyzHlJjwzIgPMYUFtWVlwcRB00MJD/mkMZSyCZ3FaqbUe0ph88xIc6HR2Gn5KUpmpBR1sNHYIaNBwYy4WsGdFSPWjlvUMRuZGRceCIf/s5YZoTlvG5gBLVxcdHKllVjeF0eaBVfCvUkcM9LX1iXc+d4SmoIfPmKm0hhdJFsjM+Vw+UNmasYFdkbWTmjhVmw+ZEa60cVLK8o68IpnJnvoIb2q6ikNWSepbTMWfyLBmavfdodWvpp6k1vaIaZwKGUXZjzpXl3UjpBds+HFle8TkUmxNxV3pUXERxrn4/gejzMiyKxu5A7QwJU5iO5Xc+HB4qMem9KcFsQjcHSkAhpLmB007slupoXG/D/XPPCD+ydTnFaLqz5mDUGJQi6J2OVAFzrHbnyRJrz0svvwEA6y6kxUlFVyx1YJId7tmrjsNj0etWVN9jekz2w0x3GC6f+TNvqyWJ6O2TsPdYFM5iF7VXDqnDv2Mptj0cnUn1kv5tVxPJmLFCUf+PcBuHX+mZfCwgmcyPXx9zqWIh/pvIxsDVhZrCSZ/zZenhnSQHDnIHPKoY4Zzhb5Ff4VZlDsIKz6bGqY2YDW+fsDK/4VZtisAiamsaPMGNQBDbP7/3CWXfKPMEPlm4ORlk9hVP93r/E6VJlj/rdHKd3mB1CtDe00fclza8EGVaJ0vY6P1BQoViMqlhGB89scrOIDXzELcFhXcxhbGpjRf33GlPjEVzyBqIGZclauZ8b/fdv/d5nZksqLMdI5L9v/4a3/AjMLt+q2c5nSG6p8qGykrlse1hniG19xnMksMy3CLcOmmwNKmLv5Oj7k0frTeN22H+M72PBYv+LclONrsQnjqVpW/kbV4f8AHefeSC51gZgAAAAASUVORK5CYII=',
      'external_logo': 'https://awsmp-logos.s3.amazonaws.com/4675c3b9-6053-4a8c-8619-6519b83bbbfd/536ec8b5c79de08fcac1086fdf74f91b.png'
  },
  default_mapping="question").plot()


In [ ]:
g.encode_point_badge(
  'assorted',
  'TopRight',
  shape="circle", #clip excess
  categorical_mapping={
      'macbook': 'laptop', #https://fontawesome.com/v4.7.0/icons/
      'Canada': 'flag-icon-ca', #ISO3611-Alpha-2: https://github.com/datasets/country-codes/blob/master/data/country-codes.csv
      'embedded_smile': 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAARkAAAC0CAMAAACXO6ihAAAAgVBMVEUAAAD////T09P4+PiYmJihoaHf39/7+/vm5uYLCwsRERHy8vJ+fn7v7+83Nze2trbZ2dlRUVFGRka9vb1xcXEYGBhbW1tWVlbq6uq7u7usrKxgYGA8PDwWFhbNzc14eHhpaWmoqKiPj48vLy8oKCggICCIiIiUlJRBQUFKSkokJCTGfDo0AAAOxklEQVR4nO1d6YKyOgxFNkFEUFwQFFEUdd7/AS9L06ZlkZm5I37i+acUSA9dkjRNJal/JPNYjfxRBlvW1guzb3leBPupJo8wLGfat0wvgaOrjEQo0b1vsfrGTKvSUiLtW7RekaRNvGRQV32L1x9it5mX0Wh86lu+3uCM24jJEPYtYT+468Kg6xuGz3cuu28Ze0FqYA7c4/Wwud3WUy3ihpq+pewBB9yTnPWMXjivfURYjxL2hIDV3j+KFx3Wwwan8TFixl5VpdtvswuWHEXGKOhBuD4xZS0mntRcPxvRYT1Lkvt0OSwLiql38qy+xP65Ar0K5nRWCgas5dZgQvWYtkEkdDzPG5gSTGeeoHkMmRDzWx+SFhx3aDGhRbXg+GmC9Y0E+pKbNJa5IfXYf6Js/eIEVW4mRuIMhKH0pztM2MvmMjtMzEh+nnC9QiP1jb6ayzgcM0PpTsSOVA4tZfQhMgNmQWsX4dwTI/1ZsvUL8G7GbYV8jplhuGjupNJKayl+7en8JNn6hUpq67WWWmJitk8SrWfArPNAR0Hdybg9R7KecQYVbtFebsuI2TxHsr5xh1mnvSEwTc9qUZTfCiHRZqwGf1WJxB7YvCQhM7utzczQ1BQ/SbDesezCDLYN5k8TrWdQZprNyT0mRm/tde8E2psaF9hmnJbX4vR7M2zo2NrQnea8MTmYARi1iKj2uhBMMx7QAiVbmnSqHSXmXHkZjAEFpF1ZtR3hUlgNpqlvWO+JBLle3DW6EMqWyMuwmOEdmXYabubzTZjy3ciFYXg4A3AOMWBRqUQwOmfCjPLA7nwzpNVOwyOV9p3cW+8HtZWXKFN0DqQZGX2L+mRMtGZe3CI4GqZ2rW9Rn45rIy+lN6aj4+8dsa6Lj9bjS3k1AUV5MOYkRhjhNaWxLmtsyfLuyZHjOMdTXZjaAHBPtSCSM3ialg5ren4M85xk+FY82uoTvVaDS1z4MOy2FfE3xuoWp6qqaiVU7TQNb+V4k1ATQhvcALRfOrJsWLzJrdiGHMmOGiKf1qDiGs93zX+0pYfCH4zjXIqvxmM+EIYy1MRRjW+mFcOwG9Z6y07KBoj+wHfEWvT9ftpMgUnw3X5U4u3XFMSNlARjX/e2GVyjibe4b8n/GBVHja9H14Wgxu3n6dbV+SQA2/deutxvBVq2aksU0QkpO9Z7O272HsdLdHigvSGXznuPMgmek6wILXObl2S2u1VtatAFlfh5UvaAL0QMzhoyWV4D2bdqx1g1shU7Ct7c04e6ko5jaWZkQcWu61tmuAzfPTgYbdbmd1LeYdhp2a/xzqBbnEa2EHwFoUeVve3DwJkGQlfmX2hMw1rRpmCjb0WBIZxZLdvC3hg0WK9mmCVX3PdWchuwAp1NqTaMBTDTg1z9YwomUI0rAaIBhuB/qYCaS1bNRVi3fW/DqAFzaDJx9doKvBLPFuolcABdrmYXytJqbk7vD+gwdRrLEfTip0v1CiCVrwv1ncGsNZzlJASTVL5uVzIoOv7u6WK9AEJS+7otpmCAD9NogtDOmqEE9gi2bv9/XwTN7QLMKXlwoQ4FnMY2swQf+EDtbND/K1sK6MQ0TGWG7RfURbcl3ZAwqOAYhITUfywkjKZB98ZgI/GAAZdbA6AG+JsvJrWBLmVvGTUTlRIzSP9DCZax04XmsWZrLNaAI4MXLL5BMZw4uasyS3YwTMcMoG0Xz7Dz+M/qg2by+Sq49C1cv9g0MeMMnBghWRNrMQOeligOdpUYf7iKDMbGE4lxBpK16SH2sY7Cy8bRepCLkg0w1Uh2DUOPInWgScfbcFkshpLi64MPPvjggw8++OCDDz744IMPPvjggw8++OCDDz744IM/wWTF0FrwHERe/ByZXgNbxQa07vc6Fou90ZunPsDAqVZaikGgljycEJtuzFxo7OdwNnJ3Y+ZOywzlQFMuqUgLMwdaZiCHU2ZII5ZmsLnUENuMeTlPrYfMnCkzncaZxNd14w12dpgddmlD0KPcab9Gse/lDbIRdmHGTIssI1634HtlQMxI0nyapnE3beY4GhQz3bH2/4iZr83ykGE5b0m2tciKLDeVeMxbXNwZtpwceAmLW/EmUp6ZW/72uGP0XhIWL7zj0WdGdqq2MbMp7ur2DoDmRb4yHo8VxZA9Jy7/3J1UQP7PMtKzIooRBZiCdeDp9ji/1dY9TyV1ozeq+W6LtRPZ5a0aqztm5hS5+dtt2UPxwXP2EBX3p+lWtgtZ9chLIT50Cfsxo2txg5Y9aEZv13KJr5FR3nXMnhayZ5Paxuwfuj/6HAjB3YpV2HAHtvPGkxKPRfJaNF717gtZfi3jlL13z8rakrlFZey4yswdJVpU2L5lljAMn5sR2viN4zJboWFXkg1P8Z6qUcrtCIlMrIST8H6UuA30SvQA9MopOxwng7fBebEV0ozPlZjwHFnd9uyXfRMyasOWHcrMkk/tGpktzHzVvPEq3WqkCHlmQvzxZU4JJ8ygw7fK04C+6s/IUXhmZG4PDmHmLFduy6FxzCjiYSqQzYkyI+5G0JqZ2dclXD6yFFoIc44Zh/s83ZhpODxIYEZIuV8yU3eAzEhkpgr9xjNTefWlkZna87GOtbvKFhwzfGfrxAzK/qfngJx3OTPLmr0l5cWCGfb9jOJWlMhqX+n2mNpTlRnbMFDXbWQGfupemgYukfUo1bRda4cNUgEyx0N1nMmZgaxkI8Ur9zLeSH1zZmZTVcPtwgucUpiCGdqijFMh9+VIap/3hvSqoa/iBupJY0T6e4EZVwtXyXIL7BHz8a4GHrxjxnGllTNAaBBmrk4QwOPdoEA+myWnq4Y+rr0NiBmbvWCtasAnYSZk/+TMgPhjmpVBZcwUwzP9lO5yBc05Z+YMnPmgIEBRMk7MqEhBkVjmwtrAjGcmIolnoHI6NQZWEb4DCthwOSDM5ICxRtBnWEeL5hM4d7ikHjbpsa1nS3YeGx2xWSYLjWdmDhOHXogfUmbW8MYY7hSYgWQ8LBXGkfuHMqNDGtMV+QOlYoQvxzFDk0tgZsD4FJiZU2L2rHZyMf1Bf2HMEDU6ZwYYRcf5NTFTDg4kw86KfWD2XIGZCSnAdvXPYYLQOWbYzlvSndC5txwzE+jpcLVoAYrahRmSKrQrMzTJHXpcAzNEmjg/C0bPVY4xX66ZGZSmCOppccyw63I3ZmgrXJ7SdLrcdWHGK4emoJC/bGVtzNygM6Gd5A3MkL69uu1myfnC3G0oOchjZo7kL9+sZ8ZrZ4bNQV41JdYDZkj1z4X8q4fMwG0+8n80MCOm1AQ1CLkhHzMD05ny9TNm0HQnT2OenW7MYLQxA6OzjFa6OjJTHWY6MLODrjn/GTP8vmY3wPt2/19mYKbDSa47MgMNGyUueMzMhPReZfMzZiTh8AwF7Wl+OWbQCNWBGTjZ9qfMzCrnihggwKswQ6eJ7zFj/JIZaVMxtuzwT5lBvrS/ZEaC3nT7KTPSLhUPT9P/ghkYgW3kAezGjPkjZjaQ1HX/Y2YyxWGq85au8wfMUNUZHSB/7MQM9UCgxImPmYHvXShHP2VGyt2w24iZ5tHld8ywI35iG5jZgXBo+oOD0Tsyg5IqPmYGTP8mHbgrMxnOMVVuSgfqz5mJqLIKpzkb0oQTtcSoGzNgV3xH07tAF4x+xswsiFzbkol/3gT3y/jwO2Ys8MpT+Qx05DWsBl9gNn7EDPjLdDatpcg/U8sM9STFP2NmWrzAoCsXnE71fWZuY3y/hNybGTN3egZQWnS2TQSlHzFzgQ/GVD3hTPUKM3TUHp1/yEzRKHXKDLxwipkRcqa2MJNQD1Ph9dgwp17uuaLCjg1HDVC6MmAmVOqZAVt75MKMP0fezlpmqBOp/KcmL3uVGei0CWKG2ktwgE+MmZH5lUT4uy4zEm0jlh5F1M9LmBH1bYGZVQKi6bNFZqSix1L3s15M+UlIbzUWixliZuQu57fkNqcj5jiviXmmaQazJxcN9rIA/4x6u+XzjXmhp96Gi90KBrKg/BYJ+AijYuWTvj/YTcxZKBf0r2jCVG+XyS+sjx+azpUVPOQ8Cr9a6jE3sG3rMjb+Z+z8mKPKXKgF/PwYkAn6w5DRqoa3ynh12A3ZkzMLc+NQp29+HPA1q6vj0TZs2caJDvGuli87UtFKfyTVQEa+4+TX8sMdPSrk2PZdWWg4l8oajWIxZqRjPTN+puKY4rKJhXOoNyzHlJjwzIgPMYUFtWVlwcRB00MJD/mkMZSyCZ3FaqbUe0ph88xIc6HR2Gn5KUpmpBR1sNHYIaNBwYy4WsGdFSPWjlvUMRuZGRceCIf/s5YZoTlvG5gBLVxcdHKllVjeF0eaBVfCvUkcM9LX1iXc+d4SmoIfPmKm0hhdJFsjM+Vw+UNmasYFdkbWTmjhVmw+ZEa60cVLK8o68IpnJnvoIb2q6ikNWSepbTMWfyLBmavfdodWvpp6k1vaIaZwKGUXZjzpXl3UjpBds+HFle8TkUmxNxV3pUXERxrn4/gejzMiyKxu5A7QwJU5iO5Xc+HB4qMem9KcFsQjcHSkAhpLmB007slupoXG/D/XPPCD+ydTnFaLqz5mDUGJQi6J2OVAFzrHbnyRJrz0svvwEA6y6kxUlFVyx1YJId7tmrjsNj0etWVN9jekz2w0x3GC6f+TNvqyWJ6O2TsPdYFM5iF7VXDqnDv2Mptj0cnUn1kv5tVxPJmLFCUf+PcBuHX+mZfCwgmcyPXx9zqWIh/pvIxsDVhZrCSZ/zZenhnSQHDnIHPKoY4Zzhb5Ff4VZlDsIKz6bGqY2YDW+fsDK/4VZtisAiamsaPMGNQBDbP7/3CWXfKPMEPlm4ORlk9hVP93r/E6VJlj/rdHKd3mB1CtDe00fclza8EGVaJ0vY6P1BQoViMqlhGB89scrOIDXzELcFhXcxhbGpjRf33GlPjEVzyBqIGZclauZ8b/fdv/d5nZksqLMdI5L9v/4a3/AjMLt+q2c5nSG6p8qGykrlse1hniG19xnMksMy3CLcOmmwNKmLv5Oj7k0frTeN22H+M72PBYv+LclONrsQnjqVpW/kbV4f8AHefeSC51gZgAAAAASUVORK5CYII=',
      'external_logo': 'https://awsmp-logos.s3.amazonaws.com/4675c3b9-6053-4a8c-8619-6519b83bbbfd/536ec8b5c79de08fcac1086fdf74f91b.png'
  },
  default_mapping="question",
  bg={'color': {'mapping': {'categorical': {'fixed': {}, 'other': 'white'}}}})\
.plot()

### Badges as continuous mappings and styling

You can also use value ranges to pick the glyph, and use text as the glyph

In [ ]:
g.encode_point_badge(
    'score',
   'TopRight',
    continuous_binning=[
        [33, None],
        [66, 'info-circle'],
        [None, 'exclamation-triangle']
    ],
    border={'width': 2, 'color': 'black', 'stroke': 'solid'},
    color={'mapping': {'categorical': {'fixed': {}, 'other': 'black'}}},
    bg={'color': {'mapping': {'continuous': {'bins': [
        [33, '#ccc'],
        [66, 'yellow'],
        [200, 'red']        
    ], 'other': 'black'}}}}).plot(render=False)


### Flag inference
The below code generates ISO3166 mappings from different conventions to Alpha-2

In [ ]:
codes = pd.read_csv('https://raw.githubusercontent.com/datasets/country-codes/master/data/country-codes.csv')
codes.columns

In [ ]:
country_to_iso_flag = {
    o['CLDR display name']: 'flag-icon-' + o['ISO3166-1-Alpha-2'].lower()
    for o in codes[['CLDR display name', 'ISO3166-1-Alpha-2']].dropna().to_dict('records')
}

# also try with   shape="circle" and   border={'width': 2, 'color': 'black', 'stroke': 'solid'}
g.encode_point_badge('origin', 'BottomRight', categorical_mapping=country_to_iso_flag).plot()